In [15]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import pickle as pk

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import  silhouette_score

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe_connected"

In [2]:
#import libraries
%matplotlib notebook
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df_apr = pd.read_csv('uber-raw-data-apr14.csv')
df_may = pd.read_csv('uber-raw-data-may14.csv')
df_jun = pd.read_csv('uber-raw-data-jun14.csv')
df_jul = pd.read_csv('uber-raw-data-jul14.csv')
df_aug = pd.read_csv('uber-raw-data-aug14.csv')
df_sep = pd.read_csv('uber-raw-data-sep14.csv')

In [4]:
df_def = pd.concat([df_apr, df_aug, df_jul, df_jun, df_may, df_sep])

In [5]:
df_def.columns = [x.lower().replace('/', '_') for x in df_def.columns]

In [6]:
df_def.date_time = pd.to_datetime(df_def['date_time'])

In [7]:
new_dates, new_times = zip(*[(d.date(), d.time()) for d in df_def['date_time']])
df_def = df_def.assign(new_date=new_dates, new_time=new_times)
df_def['hour'] = df_def.date_time.dt.hour
df_def['day'] = df_def.date_time.dt.dayofweek

In [8]:
df_def = df_def.drop(columns=["date_time"])

DROP OUTLIER

In [9]:
for i in ['lat', 'lon',]:
    df_def = df_def[~(np.abs(df_def[i]-df_def[i].mean()) > (3*df_def[i].std()))]

DETERMINATION DES COLONNES ET STANDARDSCALER

In [10]:
#data test / samedi à 20h
data_test = df_def.loc[(df_def['hour'] == 20) & (df_def['day'] == 5)]

In [11]:
data_test = data_test.drop(columns=['base', 'new_date', 'new_time'])

In [12]:
data_test.head()

,lat,lon,hour,day
6592,40.7735,-73.9599,20,5
6593,40.7110,-74.0056,20,5
6594,40.7707,-73.9620,20,5
6595,40.7154,-74.0076,20,5
6596,40.7644,-73.9719,20,5


In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [16]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(data_test.head())
X = preprocessor.fit_transform(data_test) # fit_transform !!
print('...Terminé.')
print(X[0:5, :])
print()

Preprocessing sur le train set...
          lat      lon  hour  day
6592  40.7735 -73.9599    20    5
6593  40.7110 -74.0056    20    5
6594  40.7707 -73.9620    20    5
6595  40.7154 -74.0076    20    5
6596  40.7644 -73.9719    20    5
...Terminé.
[[ 1.11333206  0.569842    0.          0.        ]
 [-0.74013632 -0.89110202  0.          0.        ]
 [ 1.03029667  0.50270891  0.          0.        ]
 [-0.60965215 -0.9550383   0.          0.        ]
 [ 0.84346706  0.18622432  0.          0.        ]]



In [17]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[47509.32541689665, 33093.03626006506, 22789.837146531103, 18749.160695900737, 15078.502811739567, 13468.91749912749, 12124.966754117253, 10929.773584663135, 10024.301279352318]


In [18]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [19]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X)
    s_score.append(silhouette_score(X, kmeans.predict(X)))

print(s_score)

[0.38660555969469607, 0.4266390949768508, 0.43387934361013825, 0.3847555209675602, 0.40939018693210993, 0.34926920608748047, 0.35565199299028427, 0.361575790563363, 0.36294565865195044]


In [20]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [21]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X)

KMeans(n_clusters=4)

In [22]:
data_test.loc[:,'Cluster_KMeans'] = kmeans.predict(X)
data_test.head()

,lat,lon,hour,day,Cluster_KMeans
6592,40.7735,-73.9599,20,5,3
6593,40.7110,-74.0056,20,5,1
6594,40.7707,-73.9620,20,5,3
6595,40.7154,-74.0076,20,5,1
6596,40.7644,-73.9719,20,5,3


In [23]:
fig = px.scatter_mapbox(
        data_test, 
        lat="lat", 
        lon="lon",
        color="Cluster_KMeans",
        mapbox_style = "carto-positron"
)

fig.show()

LUNDI TOUTES LES HEURES

In [24]:
df_lundi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 0)]

In [25]:
df_lundi = df_lundi.drop(columns=['base', 'new_date', 'new_time'])

In [26]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [27]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [28]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_lundi.head())
X_lundi = preprocessor.fit_transform(df_lundi) # fit_transform !!
print('...Terminé.')
print(X_lundi[0:5, :])
print()

Preprocessing sur le train set...
          lat      lon  hour  day
7785  40.7205 -73.9939     0    0
7786  40.7407 -74.0077     0    0
7787  40.7591 -73.9892     0    0
7788  40.7419 -74.0034     0    0
7789  40.7419 -74.0034     1    0
...Terminé.
[[-0.73576865 -0.5364991  -2.43883748  0.        ]
 [-0.07787508 -0.96454444 -2.43883748  0.        ]
 [ 0.52139432 -0.39071554 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.26114925  0.        ]]



In [29]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_lundi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[1129858.408470433, 829254.0906884766, 653796.3729775192, 536670.931910065, 454001.24636003224, 410963.0778964644, 377629.10321677104, 349135.07134922245, 327949.19692283013]


In [30]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [31]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_lundi)
    s_score.append(silhouette_score(X_lundi, kmeans.predict(X_lundi)))

print(s_score)

[0.3052534027367636, 0.34327960181750544, 0.31798711024640336, 0.3117639251540053, 0.318310413244829, 0.3231756161781162, 0.31424404969706793, 0.2965147395054782, 0.28563499384116064]


In [32]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [33]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 3)
kmeans.fit(X_lundi)

KMeans(n_clusters=4)

In [34]:
df_lundi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_lundi)
df_lundi.head()

,lat,lon,hour,day,Cluster_KMeans
7785,40.7205,-73.9939,0,0,0
7786,40.7407,-74.0077,0,0,0
7787,40.7591,-73.9892,0,0,0
7788,40.7419,-74.0034,0,0,0
7789,40.7419,-74.0034,1,0,0


In [74]:
fig = px.scatter_mapbox(df_lundi[(df_lundi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

In [ ]:
##################################################################################################

MARDI TOUTES LES HEURES

In [36]:
df_mardi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 0)]

In [37]:
df_mardi = df_mardi.drop(columns=['base', 'new_date', 'new_time'])

In [38]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [40]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_mardi.head())
X_mardi = preprocessor.fit_transform(df_mardi) # fit_transform !!
print('...Terminé.')
print(X_mardi[0:5, :])
print()

Preprocessing sur le train set...
          lat      lon  hour  day
7785  40.7205 -73.9939     0    0
7786  40.7407 -74.0077     0    0
7787  40.7591 -73.9892     0    0
7788  40.7419 -74.0034     0    0
7789  40.7419 -74.0034     1    0
...Terminé.
[[-0.73576865 -0.5364991  -2.43883748  0.        ]
 [-0.07787508 -0.96454444 -2.43883748  0.        ]
 [ 0.52139432 -0.39071554 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.26114925  0.        ]]



In [41]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_mardi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[1129852.078973231, 829254.7151290725, 653811.1901430134, 536674.7012772318, 454003.36158953945, 410957.5333199567, 377628.20721740386, 349137.2702531534, 327831.7970915941]


In [42]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [43]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_mardi)
    s_score.append(silhouette_score(X_mardi, kmeans.predict(X_mardi)))

print(s_score)

[0.30546176544978265, 0.34329197840753234, 0.31760980367310143, 0.3126649850137936, 0.3182360770667084, 0.323513996971665, 0.314234485329829, 0.2964552480220607, 0.2843478037732705]


In [44]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [45]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 3)
kmeans.fit(X_mardi)

KMeans(n_clusters=4)

In [46]:
df_mardi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_mardi)
df_mardi.head()

,lat,lon,hour,day,Cluster_KMeans
7785,40.7205,-73.9939,0,0,2
7786,40.7407,-74.0077,0,0,2
7787,40.7591,-73.9892,0,0,2
7788,40.7419,-74.0034,0,0,2
7789,40.7419,-74.0034,1,0,2


In [75]:
fig = px.scatter_mapbox(df_mardi[(df_mardi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

In [ ]:
##########################################################################

MERCREDI TOUTES LES HEURES

In [66]:
df_mercredi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 2)]

In [67]:
df_mercredi = df_mercredi.drop(columns=['base', 'new_date', 'new_time'])

In [68]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [69]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [70]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_mercredi.head())
X_mercredi = preprocessor.fit_transform(df_mercredi) # fit_transform !!
print('...Terminé.')
print(X_mercredi[0:5, :])
print()

Preprocessing sur le train set...
          lat      lon  hour  day
1011  40.7458 -73.9843     0    2
1012  40.7285 -74.0467     0    2
1013  40.7732 -73.9546     0    2
1014  40.6550 -73.9786     0    2
1015  40.7405 -74.0040     0    2
...Terminé.
[[ 0.09234206 -0.16939352 -2.60553226  0.        ]
 [-0.50584175 -2.34341013 -2.60553226  0.        ]
 [ 1.03975456  0.86535476 -2.60553226  0.        ]
 [-3.0472585   0.02919453 -2.60553226  0.        ]
 [-0.09091657 -0.85574171 -2.60553226  0.        ]]



In [71]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_mercredi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[1488650.8613862246, 1121449.103733254, 883105.4345422259, 726807.8028821521, 611639.6355937524, 556697.4886511853, 508521.34459461254, 467813.11829976604, 435785.74665506394]


In [72]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [ ]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_mercredi)
    s_score.append(silhouette_score(X_mercredi, kmeans.predict(X_mercredi)))

print(s_score)

In [ ]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 3)
kmeans.fit(X_mardi)

In [ ]:
df_mercredi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_mercredi)
df_mercredi.head()

In [ ]:
fig = px.scatter_mapbox(df_mercredi[(df_mercredi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

In [1]:
############### MODELE NON RETENU   #########################################

JEUDI TOUTES LES HEURES

In [ ]:
df_jeudi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 3)]

In [59]:
df_jeudi = df_jeudi.drop(columns=['base', 'new_date', 'new_time'])

In [60]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [61]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [62]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_jeudi.head())
X_jeudi = preprocessor.fit_transform(df_jeudi) # fit_transform !!
print('...Terminé.')
print(X_jeudi[0:5, :])
print()

Preprocessing sur le train set...
          lat      lon  hour  day
7785  40.7205 -73.9939     0    0
7786  40.7407 -74.0077     0    0
7787  40.7591 -73.9892     0    0
7788  40.7419 -74.0034     0    0
7789  40.7419 -74.0034     1    0
...Terminé.
[[-0.73576865 -0.5364991  -2.43883748  0.        ]
 [-0.07787508 -0.96454444 -2.43883748  0.        ]
 [ 0.52139432 -0.39071554 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.43883748  0.        ]
 [-0.03879229 -0.83116799 -2.26114925  0.        ]]



In [63]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_jeudi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[1129848.856102096, 829254.2332773556, 653803.9203682938, 536674.7563842073, 454001.5868243445, 410962.33896333835, 377631.4715556439, 349135.03554901923, 327975.5984857192]


In [64]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [65]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_jeudi)
    s_score.append(silhouette_score(X_jeudi, kmeans.predict(X_jeudi)))

print(s_score)

KeyboardInterrupt: 

In [ ]:
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X_jeudi)

In [ ]:
df_jeudi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_jeudi)
df_jeudi.head()

In [ ]:
fig = px.scatter_mapbox(df_jeudi[(df_jeudi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

In [ ]:
################################################################################

VENDREDI TOUTES LES HEURES

In [ ]:
df_vendredi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 4)]

In [ ]:
df_vendredi = df_vendredi.drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [ ]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_vendredi.head())
X_vendredi = preprocessor.fit_transform(df_vendredi) # fit_transform !!
print('...Terminé.')
print(X_vendredi[0:5, :])
print()

In [ ]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_vendredi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

In [ ]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [ ]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_vendredi)
    s_score.append(silhouette_score(X_vendredi, kmeans.predict(X_vendredi)))

print(s_score)

In [ ]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X_jeudi)

In [ ]:
df_vendredi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_vendredi)
df_vendredi.head()

In [ ]:
fig = px.scatter_mapbox(df_vendredi[(df_vendredi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

In [ ]:
###############################################################################

SAMEDI TOUTES LES HEURES

In [ ]:
df_samedi = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 0)]

In [ ]:
df_samedi = df_samedi.drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [ ]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_samedi.head())
X_samedi = preprocessor.fit_transform(df_samedi) # fit_transform !!
print('...Terminé.')
print(X_samedi[0:5, :])
print()

In [ ]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_samedi)
    wcss.append(kmeans.inertia_)
    
print(wcss)

In [ ]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [ ]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_samedi)
    s_score.append(silhouette_score(X_samedi, kmeans.predict(X_samedi)))

print(s_score)

In [ ]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X_samedi)

In [ ]:
df_samedi.loc[:,'Cluster_KMeans'] = kmeans.predict(X_samedi)
df_samedi.head()

In [ ]:
fig = px.scatter_mapbox(df_vendredi[(df_samedi.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

DIMANCHE TOUTES LES HEURES

In [ ]:
df_dimanche = df_def.loc[(df_def['hour'] >=0 ) & (df_def['day'] == 0)]

In [ ]:
df_dimanche = df_dimanche.drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
# Création du pipeline pour les variables quantitatives
numeric_features = [0,1,2,3] # Positions des colonnes quantitatives dans X
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)     
    ])

In [ ]:
# Preprocessings sur le dataset
print("Preprocessing sur le train set...")
print(df_dimanche.head())
X_dimanche = preprocessor.fit_transform(df_dimanche) # fit_transform !!
print('...Terminé.')
print(X_dimanche[0:5, :])
print()

In [ ]:
# Utilisation de la méthode Elbow pour trouver le nombre optimal de clusters

wcss =  []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_dimanche)
    wcss.append(kmeans.inertia_)
    
print(wcss)

In [ ]:
fig = px.line(x = range(2,11), y = wcss)
fig.show()

In [ ]:
# Utilisation du silhouette_score pour déterminer le nombre optimal de clusters
s_score = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X_samedi)
    s_score.append(silhouette_score(X_samedi, kmeans.predict(X_samedi)))

print(s_score)

In [ ]:
# Affichage de scores en fonction du nombre de clusters
fig = px.bar(x = range(2,11), y = s_score)
fig.show(renderer="iframe")

In [ ]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X_dimanche)

In [ ]:
df_dimanche.loc[:,'Cluster_KMeans'] = kmeans.predict(X_dimanche)
df_dimanche.head()

In [ ]:
fig = px.scatter_mapbox(df_dimanche[(df_dimanche.Cluster_KMeans >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="Cluster_KMeans",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()